In [ ]:
# Load the hotel reviews from CSV
import pandas as pd
import time
# importing time so the start and end time can be used to calculate file loading time
print("Loading data file now, this could take a while depending on file size")
start = time.time()
# df is 'DataFrame' - make sure you downloaded the file to the data folder
df = pd.read_csv('data/Hotel_Reviews.csv')
end = time.time()
print("Loading took " + str(round(end - start, 2)) + " seconds")


Loading data file now, this could take a while depending on file size
Loading took 2.22 seconds
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     5157

In [63]:
rows = df.shape[0]
columns = df.shape[1]

print(rows)
print(columns)

distinct_values = df["Reviewer_Nationality"].unique()
print(len(distinct_values))

most_common = df["Reviewer_Nationality"].value_counts().idxmax()
count = df["Reviewer_Nationality"].value_counts().max()
# check = []
# for i in df["Reviewer_Nationality"]:
#     if i.strip() == "United Kingdom":
#         check.append(i)    

print(f"{most_common}: {count}")
print(count)
# print(len(check))

top10 = df["Reviewer_Nationality"].value_counts().head(10)
count10 = top10.values.tolist()

print(f"{top10}: {count10}")





515738
17
227
 United Kingdom : 245246
245246
Reviewer_Nationality
United Kingdom               245246
United States of America      35437
Australia                     21686
Ireland                       14827
United Arab Emirates          10235
Saudi Arabia                   8951
Netherlands                    8772
Switzerland                    8678
Germany                        7941
Canada                         7894
Name: count, dtype: int64: [245246, 35437, 21686, 14827, 10235, 8951, 8772, 8678, 7941, 7894]


In [85]:
# Step 1: Get the top 10 nationalities
top_10_nationalities = df["Reviewer_Nationality"].value_counts().head(10).index

# Step 2: Filter dataset only for these nationalities
filtered = df[df["Reviewer_Nationality"].isin(top_10_nationalities)]

# Step 3: Group by both Reviewer_Nationality and Hotel and count visits
visits = (
    filtered.groupby(["Reviewer_Nationality", "Hotel_Name"])
    .size()
    .reset_index(name="count")
)

# Step 4: For each nationality, pick the hotel with the max count
most_visited = visits.loc[visits.groupby("Reviewer_Nationality")["count"].idxmax()]

print(most_visited)


             Reviewer_Nationality                                  Hotel_Name  \
1088                   Australia         Park Plaza Westminster Bridge London   
2498                      Canada            St James Court A Taj Hotel London   
3223                     Germany                               Hotel Da Vinci   
4175                     Ireland       Copthorne Tara Hotel London Kensington   
6002                 Netherlands                                Jaz Amsterdam   
7653                Saudi Arabia                The Cumberland A Guoman Hotel   
8334                 Switzerland                               Hotel Da Vinci   
9965        United Arab Emirates        Millennium Hotel London Knightsbridge   
10581             United Kingdom   Britannia International Hotel Canary Wharf   
12513   United States of America                               Hotel Esther a   

       count  
1088     167  
2498      61  
3223      86  
4175     239  
6002      97  
7653     142  
833

In [142]:

# unique_hotel_scores = df[['Hotel_Name', 'Average_Score']]
# print(unique_hotel_scores)

# define a function that takes a row and performs some calculation with it
def get_difference_review_avg(row):
    return row["Average_Score"] - row["Calc_Average_Score"]
   
   # 'mean' is mathematical word for 'average'
df['Calc_Average_Score'] = round(df.groupby('Hotel_Name').Reviewer_Score.transform('mean'), 1)
   
   # Add a new column with the difference between the two average scores
df["Average_Score_Difference"] = df.apply(get_difference_review_avg, axis = 1)
  
   # Create a df without all the duplicates of Hotel_Name (so only 1 row per hotel)
review_scores_df = df.drop_duplicates(subset = ["Hotel_Name"])
   
   # Sort the dataframe to find the lowest and highest average score difference
review_scores_df = review_scores_df.sort_values(by=["Average_Score_Difference"])
   
display(review_scores_df[["Average_Score_Difference", "Average_Score", "Calc_Average_Score", "Hotel_Name"]])



,Average_Score_Difference,Average_Score,Calc_Average_Score,Hotel_Name
495945,-0.8,7.7,8.5,Best Western Hotel Astoria
43688,-0.7,7.5,8.2,Mercure Paris Porte d Orleans
111027,-0.7,8.8,9.5,Hotel Stendhal Place Vend me Paris MGallery by...
178253,-0.7,7.9,8.6,Renaissance Paris Vendome Hotel
54745,-0.5,8.6,9.1,Hotel Les Rives de Notre Dame
...,...,...,...,...
478361,0.7,8.4,7.7,Hotel La Spezia Gruppo MiniHotel
22189,0.8,7.1,6.3,Holiday Inn Paris Montparnasse Pasteur
250308,0.9,8.6,7.7,MARQUIS Faubourg St Honor Relais Ch teaux
68936,0.9,6.8,5.9,Villa Eugenie


In [143]:
# Step 1: Get unique scores per hotel as a list
unique_scores = df.groupby('Hotel_Name')['Average_Score'].unique().reset_index()

# Step 2: Explode the lists so each score gets its own row
unique_scores_exploded = unique_scores.explode('Average_Score')

# Optional: sort by hotel name
unique_scores_exploded = unique_scores_exploded.sort_values('Hotel_Name')

print(unique_scores_exploded)


                              Hotel_Name Average_Score
0                     11 Cadogan Gardens           8.7
1                               1K Hotel           7.7
2     25hours Hotel beim MuseumsQuartier           8.8
3                                     41           9.6
4     45 Park Lane Dorchester Collection           9.4
...                                  ...           ...
1487            citizenM London Bankside           9.1
1488          citizenM London Shoreditch           9.1
1489            citizenM Tower of London           9.1
1490              every hotel Piccadilly           8.8
1491                   pentahotel Vienna           8.5

[1493 rows x 2 columns]


In [144]:
df['Calc_Average_Score'] = df.groupby('Hotel_Name')['Average_Score'].transform('mean')
df[['Hotel_Name', 'Average_Score', 'Calc_Average_Score']].head(10)

# Define a function to compare rounded scores
def compare_scores(row):
    avg_score_rounded = round(row['Average_Score'], 1)
    calc_score_rounded = round(row['Calc_Average_Score'], 1)
    
    if avg_score_rounded != calc_score_rounded:
        print(f"Difference found for {row['Hotel_Name']}: "
              f"Average_Score = {avg_score_rounded}, "
              f"Calc_Average_Score = {calc_score_rounded}")

# Apply the function to every row
df.apply(compare_scores, axis=1)


Difference found for Hotel Regina: Average_Score = 8.4, Calc_Average_Score = 8.5
Difference found for Hotel Regina: Average_Score = 8.4, Calc_Average_Score = 8.5
Difference found for Hotel Regina: Average_Score = 8.4, Calc_Average_Score = 8.5
Difference found for Hotel Regina: Average_Score = 8.4, Calc_Average_Score = 8.5
Difference found for Hotel Regina: Average_Score = 8.4, Calc_Average_Score = 8.5
Difference found for Hotel Regina: Average_Score = 8.4, Calc_Average_Score = 8.5
Difference found for Hotel Regina: Average_Score = 8.4, Calc_Average_Score = 8.5
Difference found for Hotel Regina: Average_Score = 8.4, Calc_Average_Score = 8.5
Difference found for Hotel Regina: Average_Score = 8.4, Calc_Average_Score = 8.5
Difference found for Hotel Regina: Average_Score = 8.4, Calc_Average_Score = 8.5
Difference found for Hotel Regina: Average_Score = 8.4, Calc_Average_Score = 8.5
Difference found for Hotel Regina: Average_Score = 8.4, Calc_Average_Score = 8.5
Difference found for Hotel R

0         None
1         None
2         None
3         None
4         None
          ... 
515733    None
515734    None
515735    None
515736    None
515737    None
Length: 515738, dtype: object

In [ ]:
num_no_negative = (df['Negative_Review'] == "No Negative").sum()
print(f"Number of rows with 'No Negative' review: {num_no_negative}")

num_no_positive = (df['Positive_Review'] == "No Positive").sum()
print(f"Number of rows with 'No Positive' review: {num_no_positive}")

both_no_reviews = df.apply(lambda x: True if x['Negative_Review'] == "No Negative" and x['Positive_Review'] == "No Positive" else False , axis=1)
print("Number of both No Negative and No Positive reviews: " + str(len(both_no_reviews[both_no_reviews == True].index)))
end = time.time()


Number of rows with 'No Negative' review: 127890
Number of rows with 'No Positive' review: 35946
Number of both No Negative and No Positive reviews: 127
Lambdas took 4908.89 seconds
